In [1]:
import sys
sys.path.append('./../src')

In [2]:
sys.path

['/Users/binhhoang/Google Drive/Data Science/Metis Bootcamp/Projects/project_4/queer_asian_stories/notebooks',
 '/Users/binhhoang/opt/anaconda3/envs/metis/lib/python38.zip',
 '/Users/binhhoang/opt/anaconda3/envs/metis/lib/python3.8',
 '/Users/binhhoang/opt/anaconda3/envs/metis/lib/python3.8/lib-dynload',
 '',
 '/Users/binhhoang/opt/anaconda3/envs/metis/lib/python3.8/site-packages',
 '/Users/binhhoang/opt/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/extensions',
 '/Users/binhhoang/.ipython',
 './../src']

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
from bs4 import BeautifulSoup
import requests
import web_scraping as ws
from importlib import reload

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
sns.set(color_codes=True)
plt.style.use('seaborn-colorblind')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 2)

In [21]:
# This piece of code saves me so much time! 
reload(ws)

<module 'web_scraping' from './src/web_scraping.py'>

## Gaysian Diaries ##
Parsing steps:
1. Get all the blog post urls by navigating through each GD page (there are currently 4 pages). Put them in a list.
2. Parse through each blog post and collect the title, post date, and the paragraphs. The
3. Save the titles, post dates in a list of dicts and the paragraphs in .txt files. 

In [4]:
gd_home_page = 'https://gaysiandiaries.com/'
response_text = requests.get(gd_home_page).text

In [5]:
gd_soup = BeautifulSoup(response_text, 'html5lib')

In [8]:
titles = []
for element in gd_soup.find_all(class_='post_title'):
    titles.append(element.text)
titles

['Diary Entry #34:\xa0What does reimagining a society that wouldn’t need the police look like?',
 'Diary Entry #33: How real are dreams?',
 'Diary Entry #32: An Ocean of Healing',
 'Diary Entry #31:\xa0Breaking Free from Helplessness',
 'Diary Entry #30:\xa0Gay, Asian, and Aspie',
 'Diary Entry #29:\xa0One Asian drag queen’s take on cultural appropriation',
 'Diary Entry #28:\xa0Realizing the Authentic Self—The Journey of a Transgendered Korean Adoptee',
 'Diary Entry #27:\xa0How do you measure a year in a life?']

In [9]:
post_dates = []
for element in gd_soup.find_all(class_='post_date'):
    post_dates.append(element.text)
post_dates

['August 1st, 2020',
 'June 12th, 2020',
 'June 9th, 2020',
 'August 1st, 2019',
 'May 15th, 2019',
 'October 15th, 2018',
 'August 15th, 2018',
 'July 15th, 2018',
 'June 1st, 2018',
 'May 1st, 2018']

In [10]:
gd_soup.find(class_='post_title').findNext().get('href')

'https://gaysiandiaries.com/post/625293600413171712/diary-entry-34-what-does-reimagining-a-society'

In [11]:
post_urls = []
for element in gd_soup.find_all(class_='post_title'):
    post_urls.append(element.findNext().get('href'))
post_urls

['https://gaysiandiaries.com/post/625293600413171712/diary-entry-34-what-does-reimagining-a-society',
 'https://gaysiandiaries.com/post/186710050569/diary-entry-33-how-real-are-dreams',
 'https://gaysiandiaries.com/post/184906201823/diary-entry-32-an-ocean-of-healing',
 'https://gaysiandiaries.com/post/179094609712/diary-entry-31-breaking-free-from-helplessness',
 'https://gaysiandiaries.com/post/177039931593/diary-entry-30-gay-asian-and-aspie',
 'https://gaysiandiaries.com/post/175936820624/diary-entry-29-one-asian-drag-queens-take-on',
 'https://gaysiandiaries.com/post/174481797837/diary-entry-28-realizing-the-authentic-selfthe',
 'https://gaysiandiaries.com/post/173497085784/diary-entry-27-how-do-you-measure-a-year-in-a']

In [12]:
# gd_34 = 'https://gaysiandiaries.com/post/625293600413171712/diary-entry-34-what-does-reimagining-a-society'
# response_text = requests.get(gd_34).text
# gd_34_soup = BeautifulSoup(response_text, 'html5lib')

In [13]:
# paragraphs = []
# for element in gd_34_soup.find_all('p')[1:]: # Start at index 1 since first paragraph is just a content warning
#     paragraphs.append(element.text)
# paragraphs

In [14]:
# with open('gaysian_diaries/gd_34.txt', 'w') as file:
#     file.write('\n\n'.join(paragraphs))     

In [15]:
# Find the jump_page class attribute in the a tag
# Find the largest number 
# Then create a function that creates a list of each page URL I need to visit while scraping 
gd_soup.find_all('a', class_='jump_page')[-1].text

'4'

In [6]:
gd_blog_urls = ws.gd_get_blog_urls(gd_home_page)

In [7]:
len(gd_blog_urls)

34

In [8]:
gd_blog_urls

['https://gaysiandiaries.com/post/625293600413171712/diary-entry-34-what-does-reimagining-a-society',
 'https://gaysiandiaries.com/post/186710050569/diary-entry-33-how-real-are-dreams',
 'https://gaysiandiaries.com/post/184906201823/diary-entry-32-an-ocean-of-healing',
 'https://gaysiandiaries.com/post/179094609712/diary-entry-31-breaking-free-from-helplessness',
 'https://gaysiandiaries.com/post/177039931593/diary-entry-30-gay-asian-and-aspie',
 'https://gaysiandiaries.com/post/175936820624/diary-entry-29-one-asian-drag-queens-take-on',
 'https://gaysiandiaries.com/post/174481797837/diary-entry-28-realizing-the-authentic-selfthe',
 'https://gaysiandiaries.com/post/173497085784/diary-entry-27-how-do-you-measure-a-year-in-a',
 'https://gaysiandiaries.com/post/172499454622/diary-entry-26-my-firsts',
 'https://gaysiandiaries.com/post/171431776398/diary-entry-25-lake-town',
 'https://gaysiandiaries.com/post/170399491614/diary-entry-24-my-experience-being-queer-asian',
 'https://gaysiandiar

In [9]:
gd_1_url = 'https://gaysiandiaries.com/post/140302789153/diary-entry-1'

In [11]:
gd_34_url = 'https://gaysiandiaries.com/post/625293600413171712/diary-entry-34-what-does-reimagining-a-society'

In [12]:
ws.gd_get_blog_title(gd_1_url)
ws.gd_get_blog_title(gd_34_url)

'Diary Entry #1'

'Diary Entry #34:\xa0What does reimagining a society that wouldn’t need the police look like?'

In [15]:
ws.gd_get_blog_date(gd_1_url)
ws.gd_get_blog_date(gd_34_url)

'March 1st, 2016'

'August 1st, 2020'

In [19]:
print(ws.gd_get_blog_text(gd_1_url))

Dear Diary,

I spend a lot of time analyzing how much our past experiences affect who we are today. Growing up, my father wasn’t around much. He did business internationally while my mom took care of us at home. My parents were always supportive of me and my sisters. My dad was endearing but was never a man of many words. My mother cared for us so much and always wanted the best for us. Every night before bed, my mom would hold my hands and tell me she loved me. She made sure I knew that every night before I fell asleep.

When I was 8, my mom found out my father was having an affair during his Asia business trips. I don’t know how she found out, but she somehow obtained a photo of the mistress and some letters that my dad had written to her. I noticed my mom staying up late at night crying, obsessively staring at the evidence that my father was being unfaithful. 

My mom let us know what my dad was doing and told us it was not ok. It hurt us so much to see our mom crying every night th

In [18]:
print(ws.gd_get_blog_text(gd_34_url))

Eight years ago, I lived in Flushing and had recently learned the ways of Korean drumming. One day, I was recruited as a drummer to go to Washington, DC to march and raise awareness about immigration legislation. We left New York City at around 6 am and got back in the evening.

I have a habit of making stupid mistakes that get me into a bit of trouble all the time. When I’ve been tired, hungry, cold, or thirsty, I’ve been even prone to unintentionally cause myself harm. That evening, I was tired as I was walking home alone after all the activities with the contingent. There were around 5 Black or Brown kids hanging around that I had to pass by on my way home. I took a glance at them but avoided eye contact.

What you should know at this point is that I’m also trans and at that time I had a fraction of the confidence I have now. There was also no man in my life that had ever thought to teach me how to physically conduct myself.

Maybe it was because these kids also needed to assert the

In [22]:
gd_blog_dicts = ws.gd_get_blog_dicts(gd_home_page)

In [33]:
gd_blog_dicts[-2]

{'title': 'Diary Entry #2',
 'date': 'April 1st, 2016',
 'text': 'Dear Diary,\n\nTen years ago, I took one step out of the closet to my mother.\n\nI was 21, and I just graduated from college. I was out, but only to my friends. My family—including my mother, father, and sister—and a few of my parents’ friends decided to take me out to dinner at a Chinese restaurant in Edison, New Jersey.\xa0\n\nI remember the night well. My mother’s friend, Carol (name changed), sat next to me at one of those large, round tables with the rotating tray in the center. I was exhausted after a long graduation ceremony, and I just wanted to get through dinner so I could go home and sleep. To my chagrin, Carol asked me several times during dinner why I hadn’t found a girlfriend yet. Usually, I would just laugh and redirect the conversation, but I had no energy, so I just bluntly and loudly said “I’m just not interested.” \n\nMy mother, noticing the change, quickly changed the subject.\n\nI often wonder what w

In [41]:
reload(ws)

<module 'web_scraping' from './src/web_scraping.py'>

In [34]:
with open('pickles/gd_blog_dicts.pickle', 'wb') as to_write:
    pickle.dump(gd_blog_dicts, to_write)

In [5]:
g3s_post_directory_page = 'https://gaysianthirdspace.tumblr.com/tags'
g3s_soup = ws.create_soup(g3s_post_directory_page)

In [6]:
# g3s_soup.find('div', class_='body-text').find_all('a')[-1].get('href')

In [7]:
ws.g3s_get_tag_page_url(g3s_post_directory_page)

['http://gaysianthirdspace.tumblr.com/tagged/bodyimage',
 'http://gaysianthirdspace.tumblr.com/tagged/career',
 'http://gaysianthirdspace.tumblr.com/tagged/comingout',
 'http://gaysianthirdspace.tumblr.com/tagged/community',
 'http://gaysianthirdspace.tumblr.com/tagged/dating',
 'http://gaysianthirdspace.tumblr.com/tagged/family',
 'http://gaysianthirdspace.tumblr.com/tagged/followerfriday',
 'http://gaysianthirdspace.tumblr.com/tagged/friendship',
 'http://gaysianthirdspace.tumblr.com/tagged/heritage',
 'http://gaysianthirdspace.tumblr.com/tagged/identity',
 'http://gaysianthirdspace.tumblr.com/tagged/q&a',
 'http://gaysianthirdspace.tumblr.com/tagged/relationships',
 'http://gaysianthirdspace.tumblr.com/tagged/romance',
 'http://gaysianthirdspace.tumblr.com/tagged/school']